BDLE 2022

date du document  :  16/09/2022

# TP2 BDLE


SQL avec fonctions sur des fenêtres

## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 63.3 MB/s 


Démarrer la session spark

In [ ]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1665313642052


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = r.lower().startswith('select')
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table


## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [ ]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [ ]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

On considère un fichier décrivant les check-ins d'utilisateurs et leur déplacements. Une ligne correspond à une photo prise par un utilisateur. On connait le lieu et la date d'une photo. 
Les photos consécutives durant une journée forment une séquence dont on connait l'identifiant (seqID). 

Lire les 2 premières lignes du  fichier csv en python. 
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [ ]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [ ]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Les POI avec la plus grande frequence poiFreq

In [ ]:
%%sql
select distinct poiID, poiFreq
from user_visits
order by poiFreq desc

,poiID,poiFreq
0,11,4142
1,22,3619
2,21,3594
3,16,3553
4,1,3506
5,4,3056
6,7,2064
7,23,1874
8,8,1736
9,25,1701


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [ ]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [ ]:
%%sql
cache table POI;

SELECT * 
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [ ]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Identifier les thèmes

Définir la table Theme1(theme_id, name) avec les numéros de thème commençant à 1 pour les noms de thème triés dans l'ordre croissant.

In [ ]:
%%sql
create or replace temp view Theme1 as

select  rank() over (order by theme) as theme_id , theme as name
from themes ;

select * from Theme1

,theme_id,name
0,1,Amusement
1,2,Beach
2,3,Cultural
3,4,Shopping
4,5,Sport
5,6,Structure


#### 2) Classement des séquences par leur plus grand nombre de POI **distincts**

Définir la table TopSeq(seqID, nbPOI, rang) avec rang=1 pour la séquence ayant le plus grand nbPOI.


In [ ]:
%%sql

create or replace temp view TopSeq as

select seqID, count(distinct poiID),rank() over ( order by count(distinct poiID) desc) as rang
from user_visits
group by seqID;


select * from TopSeq

,seqID,count(DISTINCT poiID),rang
0,298,13,1
1,4961,10,2
2,4351,9,3
3,5964,9,3
4,5369,8,5
...,...,...,...
95,5183,4,60
96,5539,4,60
97,813,4,60
98,2340,4,60


#### 2) Identifier les check-ins




2a) Définir la table Visite1 telle qu'il n'y pas de doublons sur le triplet (seqID, poiID, date)

In [ ]:
%%sql
create or replace temp view Visite1 as
select seqID, poiID, date
from user_visits
Group by  seqID, poiID, date
;



select *
from Visite1
where seqID in (298, 510)
order by seqID, date, poiID;

,seqID,poiID,date
0,298,22,1371514467
1,298,7,1371514471
2,298,7,1371514472
3,298,23,1371516913
4,298,28,1371516914
5,298,28,1371516915
6,298,1,1371517315
7,298,29,1371517318
8,298,30,1371517318
9,298,30,1371517319


2b) Montrer qu'il existe des séquence où une même date est associée à plusieurs POI distincts

In [ ]:
%%sql
select seqID, date , count(distinct(poiID)) as nb_POI_meme_date
from Visite1
group by seqID, date
having count(distinct(poiID)) >= 2
order by  nb_POI_meme_date desc, SeqID ; 


,seqID,date,nb_POI_meme_date
0,816,1291341600,3
1,822,1292896800,3
2,905,1317952800,3
3,1225,228664800,3
4,4271,1092646926,3
5,5497,1169172000,3
6,271,778341600,2
7,298,1371519708,2
8,298,1371519010,2
9,298,1371517318,2


2c) A partir de la table précédente, définir la table Visite2(seqID, poiID, date, num) avec *num* étant le numéro d'ordre dans une séquence.
Les POI visités à une même date sont triés dans l'ordre croissant de poiID.

In [ ]:
%%sql

create or replace temp view Visite2 as
select seqID, poiID, date , rank() over (partition by seqID order by date, poiID) as num
from Visite1

;

select * from Visite2
where seqID in (298, 510)
order by seqID, num

,seqID,poiID,date,num
0,298,22,1371514467,1
1,298,7,1371514471,2
2,298,7,1371514472,3
3,298,23,1371516913,4
4,298,28,1371516914,5
5,298,28,1371516915,6
6,298,1,1371517315,7
7,298,29,1371517318,8
8,298,30,1371517318,9
9,298,30,1371517319,10


#### 3) Identifier les Visites de POI

Définir la table Visite3(userID, seqID, poiID, poiPosition) telle que la  poiPosition vaut i pour le ième POI visité dans une séquence.

Indications:

La visite d'un POI correspond à toutes les photos consécutives prises à ce POI.

Un même POI peut apparaitre plusieurs fois à des positions différentes dans une séquence si au moins un autre POI a été visité entre temps dans la séquence.


##### 3a) POI précédent
Commencer par associer chaque événement avec le POI précédent dans la séquence

In [ ]:
%%sql
create or replace temp view Visite3a as

select seqID, date, poiID, num, first(poiID) OVER(ORDER BY seqID,date ROWS BETWEEN 1 PRECEDING AND  current row ) as precedant
from Visite2;
 

select * from Visite3a
where seqID in (298, 510)
order by seqID, date;


,seqID,date,poiID,num,precedant
0,298,1371514467,22,1,21
1,298,1371514471,7,2,22
2,298,1371514472,7,3,7
3,298,1371516913,23,4,7
4,298,1371516914,28,5,23
5,298,1371516915,28,6,28
6,298,1371517315,1,7,28
7,298,1371517318,29,8,1
8,298,1371517318,30,9,29
9,298,1371517319,30,10,30


##### 3b) Début de visite
Ajouter un attribut *début* valant 1 pour le premier élément d'une séquence et 0 sinon. 
Indication, penser à la syntaxe case when ...

In [ ]:
%%sql
create or replace temp view Visite3b as

select seqID, date, poiID, num, precedant, 
CASE
    WHEN precedant = poiID THEN 0
    ELSE 1
END AS debut
from Visite3a;


select * from Visite3b
where seqID in (298, 510)
order by seqID, date;

,seqID,date,poiID,num,precedant,debut
0,298,1371514467,22,1,21,1
1,298,1371514471,7,2,22,1
2,298,1371514472,7,3,7,0
3,298,1371516913,23,4,7,1
4,298,1371516914,28,5,23,1
5,298,1371516915,28,6,28,0
6,298,1371517315,1,7,28,1
7,298,1371517318,29,8,1,1
8,298,1371517318,30,9,29,1
9,298,1371517319,30,10,30,0


##### 3c) Ordonner les POI visités
Définir la table Visite3 décrite au début de la question 3). Ajouter l'attribut poiPosition 

In [ ]:
%%sql
create or replace temp view Visite3 as

select seqID, date,poiID, num, sum(debut) over (partition by seqID order by num) as poiPosition
from Visite3b;


select * from Visite3
where seqID in (298, 510)
order by seqID, date, poiID;

,seqID,date,poiID,num,poiPosition
0,298,1371514467,22,1,1
1,298,1371514471,7,2,2
2,298,1371514472,7,3,2
3,298,1371516913,23,4,3
4,298,1371516914,28,5,4
5,298,1371516915,28,6,4
6,298,1371517315,1,7,5
7,298,1371517318,29,8,6
8,298,1371517318,30,9,7
9,298,1371517319,30,10,7


#### 4) Durée de visite d'un POI
Définir la table Visite4(seqID, poiPosition, poiID, duree) comme étant la différence entre la plus grande et la plus petite date des evenements consécutifs associées à un même POI.

##### durée de visite :
seqId : date : poiID
  1      1       1       
  1      2       1       durée = 4-1 = 3
  1      4       1
  1      10      3       durée = 0

In [ ]:
def my_func(liste):
  return liste[-1] - liste[0]

spark.udf.register("my_func", my_func)


<function __main__.my_func(liste)>

In [ ]:
%%sql

create or replace temp view Visite4 as

SELECT seqID, poiPosition, my_func(collect_list(date)) as duree
FROM Visite3
GROUP BY seqID, poiPosition;

select * from Visite4
where seqID in (298, 510)
order by seqID, poiPosition

,seqID,poiPosition,duree
0,298,1,0
1,298,2,1
2,298,3,0
3,298,4,1
4,298,5,0
5,298,6,0
6,298,7,2
7,298,8,0
8,298,9,0
9,298,10,0


#### 5) Nombre de visites sur une semaine glissante
a) Définir la table Visite5a(userID, annee, mois,jour, nbVisite) : nbVisite est le nombre de visites qu'un utilisateur a fait chaque jour. 



In [ ]:
%%sql

create or replace temp view Visite5a as 
select userID, year(TIMESTAMP(date)) as year,
               month(TIMESTAMP(date)) as month,
               day(TIMESTAMP(date)) as day,
               count(*) as nbVisite
from user_visits
group by userID, year(TIMESTAMP(date)), month(TIMESTAMP(date)), day(TIMESTAMP(date));

select *
from Visite5a;


,userID,year,month,day,nbVisite
0,10012675@N05,2011,10,23,8
1,10014440@N06,2008,10,5,6
2,10116041@N02,2011,9,20,1
3,101330524@N02,2008,9,17,1
4,10282509@N00,2014,3,23,11
...,...,...,...,...,...
95,19761391@N06,2011,2,23,26
96,19761391@N06,2012,4,28,1
97,20532289@N00,2007,9,10,2
98,20634971@N00,2006,8,18,1


b) En déduire la table Visite5b(userID, anne, mois, jour, nbVisite7jours) : nbVisite7jours étant le nombre de visites effectuées sur une semaine glissante.

In [ ]:
%%sql

create or replace temp view Visite5b as 
select userID, year, month, day, nbVisite, sum(nbVisite) OVER (PARTITION BY userID, year, month
                                                         ORDER BY day
                                                         range BETWEEN 6 PRECEDING AND CURRENT ROW) as nbVisite7jours
from Visite5a
order by userID, year, month, day
;

select *
from Visite5b;

,userID,year,month,day,nbVisite,nbVisite7jours
0,10007579@N00,2012,9,5,1,1
1,10012675@N05,2006,3,19,4,4
2,10012675@N05,2006,3,21,1,5
3,10012675@N05,2011,10,22,1,1
4,10012675@N05,2011,10,23,8,9
...,...,...,...,...,...,...
95,10627620@N06,2010,6,22,1,1
96,10627620@N06,2010,11,5,1,1
97,10627620@N06,2011,11,8,151,151
98,10627620@N06,2011,12,3,27,27


#### 6) Déplacements entre deux POI
Définir la table Duree_Deplacement(seqID, poiPosition, poiID, deplacement). *deplacement* est la durée depuis la fin de la visite du poi courant jusqu'au début de visite du prochain POI dans une séquence.


In [ ]:
%%sql
create or replace temp view Visite6 as

select seqID, poiPosition, poiID, date, sum(deplacement) over (partition by seqID,poiID,poiPosition order by deplacement desc) as deplacements
from (select seqID, poiPosition, poiID, date, 
      last_value(date) over (order by date rows between current row and 1 following)-date as deplacement from Visite3 order by date )
order by seqID, poiID
;



select * from Visite6
where seqID in (298, 510)
order by seqID, date, poiID;

-- j'ai d'abord calculé les difference des dates entre chaque valeur de date et la valeur de la date qui suit juste aprés, dans le cas de la derniere date d'une partititon c'est la date courante - la premiere date d'un poi 
-- puis j'ai fait l'addition pour une partition donnée (seqID, poiPoisition, poiID) la somme de la valeur courante et qui la suit, dans le cas de la derniére date d'une partiton c'est la somme de cette date + 0

,seqID,poiPosition,poiID,date,deplacements
0,298,1,22,1371514467,4
1,298,2,7,1371514471,2442
2,298,2,7,1371514472,2441
3,298,3,23,1371516913,1
4,298,4,28,1371516914,401
5,298,4,28,1371516915,400
6,298,5,1,1371517315,3
7,298,6,29,1371517318,0
8,298,7,30,1371517318,430
9,298,7,30,1371517319,430
